# App for running GECCO jobs in Galaxy

1. Upload and run workflow.
2. Monitor the job.
3. Receive completion notification with some basic summary provided by Galaxy.

Note: "Receiving" the results (tentatively download) is part of the analysis pipeline.

In [1]:
import os
import sys
import json
from datetime import datetime
from platform import python_version
import logging

# Import
import bioblend.galaxy as g  # BioBlend is a Python library, wrapping the functionality of Galaxy and CloudMan APIs
# import boto3
import pandas as pd
import panel as pn
from bioblend.galaxy import GalaxyInstance
from bioblend.galaxy.config import ConfigClient


# instead of the jupyter magic, you can also use
from dotenv import load_dotenv
load_dotenv()
# %load_ext dotenv
# %dotenv

True

In [2]:

FORMAT = "%(asctime)s | %(levelname)s | %(name)s | %(message)s"

@pn.cache
def reconfig_basic_config(format_=FORMAT, level=logging.INFO):
    """(Re-)configure logging"""
    logging.basicConfig(format=format_, level=level, force=True)
    logging.info("Logging.basicConfig completed successfully")

In [3]:
class BCGalaxy:
    def __init__(self, url_var_name, api_key_var_name):
        self.logger = logging.getLogger('BCGalaxy')
        self.logger.setLevel(logging.DEBUG)
        # logging.debug("test")

        # url is a name of the variable saved in the .env file
        # the same for the api_key
        try:
            self.set_galaxy_env(url_var_name, api_key_var_name)
            self.cfg = ConfigClient(self.gi)
            self.logger.info(f'User: {self.cfg.whoami()}')
            self.logger.info(f'Galaxy version: {self.cfg.get_version()}')
        except TypeError:
            self.logger.error("Please provide valid Galaxy URL and API key!")
        except Exception as e:
            self.logger.error(f"Error: {e}")
    
    ###########
    # Getters #
    ###########
    def set_galaxy_env(self, url_var_name, api_key_var_name):
        self.url = os.getenv(url_var_name)
        self.api_key = os.getenv(api_key_var_name)
        self.gi = GalaxyInstance(self.url, key=self.api_key)

    def get_datasets_by_key(self, key, value):
        lst_dict = [k for k in self.gi.datasets.get_datasets() if 
                    key in k and
                    k[key] == value]

        self.ds_names = [k["name"] for k in lst_dict]
        return self.ds_names
    
    # def get_datasets_id(self, name):
    #     files = [k["id"] for k in self.gi.datasets.get_datasets() if k["name"] == name]
    #     if len(files) > 1:
    #         self.logger.warning(f"Multiple datasets with the same name: {name}")
    #     self.dataset_id = files[0]
    #     return self.dataset_id

    def get_datasets(self, name=None):
        self.dataset_lst = self.gi.show_matching_datasets(self.history_id, name_filter=name)
        self.ds_names = [k["name"] for k in self.dataset_lst]
    
    def get_histories(self):
        self.histories = self.gi.histories.get_histories()

    ###########
    # Setters #
    ###########
    def set_tool(self, tool_id):
        self.tool_id = tool_id
        self.logger.info(f"Tool Info: {self.gi.tools.show_tool(self.tool_id)}")

    def set_history(self, create: bool = True, hid=None, hname=None):
        # name is id or the name of the newly created history
        if create:
            if hname is None:
                hname = f"History created at {datetime.now()}"
            self.history_name = hname
            self.history_id = self.gi.histories.create_history(hname)['id']
            self.logger.info(
                f"History Info: {self.gi.histories.show_history(self.history_id)}")
        else:  # user wants to use the existing history
            self.history_id = hid
            self.history_name = self.gi.histories.show_history(self.history_id)['name']

    def set_dataset(self, dataset_id):
        self.dataset_id = dataset_id
        self.logger.info(f"Dataset Info: {self.gi.datasets.show_dataset(self.dataset_id)}")

    ########
    # Jobs #
    ########
    def show_job_status(self, job_id):
        raise NotImplementedError

    ###########
    # Actions #
    ###########
    def upload_file(self, file_path):
        upload = self.gi.tools.upload_file(file_path, self.history_id)
        self.set_dataset(upload['outputs'][0]['id'])

In [4]:
exp = BCGalaxy("GALAXY_EARTH_URL", "GALAXY_EARTH_KEY")

In [7]:
## debugging

# exp.get_histories()
# exp.histories

# exp.set_history(create=False, hid="8d8d4bf21253beda")

## Environment setup

In [38]:
# setup the galaxy environment

In [41]:
if 'google.colab' in str(get_ipython()):
    root_folder = os.path.abspath(os.path.join('/content/momics-demos'))
    # data_folder = os.path.join(root_folder, 'parquet_files')
else:
    root_folder = os.path.abspath(os.path.join('../'))

assets_folder = os.path.join(root_folder, 'assets')

## App setup

In [37]:
import panel as pn

In [9]:
def handle_login(clicks):
    exp = BCGalaxy("GALAXY_EARTH_URL", "GALAXY_EARTH_KEY")
    pn.state.notifications.info(f"User logged in: {exp.cfg.whoami()}")
    logger.info(f'You have clicked me {clicks} times')

In [8]:
# buttons
but_login = pn.widgets.Button(name="🔐 Galaxy Login")

In [ ]:
# histories
get_histories = pn.widgets.Button(name="📚 Get Histories")

In [ ]:
reconfig_basic_config()
logger = logging.getLogger(name="app")

pn.extension("tabulator")
pn.extension(notifications=True)
ACCENT = "teal"

styles = {
    "box-shadow": "rgba(50, 50, 93, 0.25) 0px 6px 12px -2px, rgba(0, 0, 0, 0.3) 0px 3px 7px -3px",
    "border-radius": "4px",
    "padding": "10px",
}

# TODO: there is a bug in the panel library that does not allow to open png files, renoming does not help 
image = pn.pane.JPG(os.path.join(assets_folder, "figs/metaGOflow_logo_italics.jpg"),
                    width=200, height=100)




template = pn.template.FastListTemplate(
    title="Run GECCO on Galaxy",
    sidebar=[image,
             but_login, pn.bind(handle_login, clicks=but_login.param.clicks),
             pn.layout.Divider(),
             
             ],
    main=[pn.Column("## Histories", 
                    # get_histories, pn.bind(handle_login, clicks=but_login.param.clicks), select_history,
                    "## Datasets", #get_datasets, select_dataset,
                    sizing_mode="stretch_both",
                   )],
    main_layout=None,
    accent=ACCENT,
    meta_refresh="2",
)
logger.info("Template created")
template.servable()